In [0]:
start_from_pickle = True

In [0]:
import pickle

if not start_from_pickle:
    with open(
        "/Workspace/Users/wilson.kan@neofinancial.com/pkls/appl_neo_v2_modeling_ready.pkl",
        "wb",
    ) as f:  # open a text file
        pickle.dump(modeling_dummy_df, f)
else:
    with open(
        "/Workspace/Users/wilson.kan@neofinancial.com/pkls/appl_neo_v2_modeling_ready.pkl",
        "rb",
    ) as f:  # Correctly opening the file in binary read mode
        modeling_dummy_df = pickle.load(f)

In [0]:
import numpy as np

modeling_oot = modeling_dummy_df[
    modeling_dummy_df["month_end"].isin(
        [np.datetime64("2023-06-30"), np.datetime64("2023-07-31")]
    )
]
modeling_intime = modeling_dummy_df[
    modeling_dummy_df["month_end"] < np.datetime64("2023-06-30")
]
modeling_intime = modeling_dummy_df[
    modeling_dummy_df["month_end"] > np.datetime64("2022-12-31")
]

In [0]:
#Modeling only thick

modeling_oot = modeling_oot[modeling_oot["thin"] == 1]
modeling_intime = modeling_intime[modeling_intime["thin"] == 1]

In [0]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split

# dropping duplicated features
X_train, X_test, y_train, y_test = train_test_split(
    modeling_intime.drop(["month_end", "isdefault_1y", "originalCreditScore", "GO17"], axis=1),
    modeling_intime["isdefault_1y"],
    test_size=0.2, random_state=459339
)
# create model instance
bst = XGBClassifier(
    n_estimators=100, max_depth=6, colsample_bytree = 0.75, subsample = 0.5, gamma = 1, eta = 0.1, min_child_weight = 2, objective="binary:logistic", random_state=939180
)
# fit model
bst.fit(X_train, y_train, verbose=True, early_stopping_rounds=10, eval_metric="auc", eval_set=[(X_test, y_test)])
xgb.plot_importance(bst)
# make predictions
# preds = bst.predict(X_test)

In [0]:
feature_imp = []

for i in range(len(bst.feature_importances_)):
    feature_imp.append((bst.feature_names_in_[i], bst.feature_importances_[i]))

feature_imp.sort(key=lambda tup: tup[1], reverse=True)
feature_imp

In [0]:
top_set = []
for i in range(20):
  top_set.append(feature_imp[i][0])
print(top_set)
# create model instance
bst = XGBClassifier(
    n_estimators=100, max_depth=6, colsample_bytree = 0.75, subsample = 0.5, gamma = 1, eta = 0.1, min_child_weight = 2, random_state=4916
)
# fit model
bst.fit(X_train[top_set], y_train, early_stopping_rounds=10, eval_metric="auc", eval_set=[(X_test[top_set], y_test)])
xgb.plot_importance(bst)

In [0]:
from sklearn.metrics import roc_auc_score

x_train_scr = bst.predict_proba(X_train[top_set])
x_test_scr = bst.predict_proba(X_test[top_set])
x_oot_scr = bst.predict_proba(modeling_oot[top_set])
print("train", 2*roc_auc_score(y_train, x_train_scr[:, 1])-1)
print("test", 2*roc_auc_score(y_test, x_test_scr[:, 1])-1)
print("oot", 2*roc_auc_score(modeling_oot["isdefault_1y"], x_oot_scr[:, 1])-1)

In [0]:
# sp = modeling_dummy_df[modeling_dummy_df["subprime"] == 1]
thin = modeling_dummy_df[modeling_dummy_df["thin"] == 1]
# thick = modeling_dummy_df[
#     modeling_dummy_df["subprime"] + modeling_dummy_df["thin"] == 0
# ]
# x_subprime_scr = bst.predict_proba(sp[top_set])
x_thin_scr = bst.predict_proba(thin[top_set])
# x_thick_scr = bst.predict_proba(thick[top_set])
# print("subprime", 2*roc_auc_score(sp["isdefault_1y"], x_subprime_scr[:, 1])-1)
print("thin", 2*roc_auc_score(thin["isdefault_1y"], x_thin_scr[:, 1])-1)
# print("thick", 2*roc_auc_score(thick["isdefault_1y"], x_thick_scr[:, 1])-1)